# Search: Solving a Maze Using a Goal-based Agent

## Instructions

Total Points: Undergrads 100 / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a PDF file. 


## Introduction

The agent has a map of the maze it is in and the environment is assumed to be **deterministic, discrete, and known.** The agent must use the map to plan a path through the maze from the starting location $S$ to the goal location $G$.  This is a planing exercise for a goal-based agent, so you do not need to implement an environment, just use the map to search for a path. Once the plan is made, the agent in a deterministic environment (i.e., the transition function is deterministic with the outcome of each state/action pair fixed and no randomness) can just follow the path and does not need to care about the percepts.
This is also called an **[open-loop system](https://en.wikipedia.org/wiki/Open-loop_controller).**
The execution phase is trivial and we do not implement it in this exercise.

Tree search algorithm implementations that you find online and used in general algorithms courses have often a different aim. These algorithms assume that you already have a tree in memory. We are interested in dynamically creating a search tree with the aim of finding a good/the best path from the root node to the goal state. Follow the pseudo code presented in the text book (and replicated in the slides) closely. Ideally, we would like to search only a small part of the maze, i.e., create a search tree with as few nodes as possible. 

Several mazes for this exercise are stored as text files. Here is the small example maze:

In [1]:
with open("small_maze.txt", "r") as f:
    maze_str = f.read()
print(maze_str)

XXXXXXXXXXXXXXXXXXXXXX
X XX        X X      X
X    XXXXXX X XXXXXX X
XXXXXX     S  X      X
X    X XXXXXX XX XXXXX
X XXXX X         X   X
X        XXX XXX   X X
XXXXXXXXXX    XXXXXX X
XG         XX        X
XXXXXXXXXXXXXXXXXXXXXX



__Note:__ The mazes above contains cycles and therefore the state space may not form proper trees unless cycles are prevented. Therfore, you will need to deal with cycle detection in your code.

## Parsing and pretty printing the maze

The maze can also be displayed in color using code in the module [maze_helper.py](maze_helper.py). The code parses the string representing the maze and converts it into a `numpy` 2d array which you can use in your implementation. Position are represented as a 2-tuple of the form `(row, col)`. 

In [2]:
import maze_helper as mh

maze = mh.parse_maze(maze_str)

# look at a position in the maze by subsetting the 2d array
print("Position(0,0):", maze[0, 0])

# there is also a helper function called `look(maze, pos)` available
# which uses a 2-tuple for the position.
print("Position(8,1):", mh.look(maze, (8, 1)))

Position(0,0): X
Position(8,1): G


A helper function to visualize the maze is also available.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# use higher resolution images in notebook

mh.show_maze(maze)

Find the position of the start and the goal using the helper function `find_pos()`

In [ ]:
print("Start location:", mh.find_pos(maze, what = "S"))
print("Goal location:", mh.find_pos(maze, what = "G"))

Helper function documentation.

In [ ]:
help(mh)

## Tree structure

Here is an implementation of the basic node structure for the search algorithms (see Fig 3.7 on page 73). I have added a method that extracts the path from the root node to the current node. It can be used to get the path when the search is completed.

In [3]:
class Node:
    def __init__(self, pos, parent, action, cost):
        self.pos = tuple(pos)    # the state; positions are (row,col)
        self.parent = parent     # reference to parent node. None means root node.
        self.action = action     # action used in the transition function (root node has None)
        self.cost = cost         # for uniform cost this is the depth. It is also g(n) for A* search

    def __str__(self):
        return f"Node - pos = {self.pos}; action = {self.action}; cost = {self.cost}"
    
    def get_path_from_root(self):
        """returns nodes on the path from the root to the current node."""
        node = self
        path = [node]
    
        while not node.parent is None:
            node = node.parent
            path.append(node)
        
        path.reverse()
        
        return(path)

If needed, then you can add more fields to the class like the heuristic value $h(n)$ or $f(n)$.

Examples for how to create and use a tree and information on memory management can be found [here](../Python_Code_Examples/trees.ipynb).

# Tasks

The goal is to:

1. Implement the following search algorithms for solving different mazes:

    - Breadth-first search (BFS)
    - Depth-first search (DFS)
    - Greedy best-first search (GBFS)
    - A* search

2. Run each of the above algorithms on the 
    - [small maze](small_maze.txt), 
    - [medium maze](medium_maze.txt), 
    - [large maze](large_maze.txt), 
    - [open maze](open_maze.txt),
    - [wall maze](wall_maze.txt),
    - [loops maze](loops_maze.txt),
    - [empty maze](empty_maze.txt), and
    - [empty 2_maze](empty_2_maze.txt).
    
3. For each problem instance and each search algorithm, report the following in a table:

    - The solution and its path cost
    - Total number of nodes expanded
    - Maximum tree depth
    - Maximum size of the frontier

4. Display each solution by marking every maze square (or state) visited and the squares on the final path.

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement inteligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formated professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.

## Task 1: Defining the search problem and determining the problem size [10 Points]

Define the components of the search problem:

Use verbal descriptions, variables and equations as appropriate. 

* Initial state = The initial state is the starting position of the agent within the maze.  As noted above, this can be found and printed to the console with the simple print statement: print("Start location:", mh.find_pos(maze, what = "S")).
* Actions = Actions are defined as each individual movement taken by the agent as it moves from the initial state to the goal state.  Actions contain all of the different moves taken down incorrect paths as well as the final correct path and anything in between.  The sum of all actions represents the overall efficiency of each model.
* Transition model = The transition model is the returned state of the agent and result of each individual action taken throughout the runtime of the algorithm.  This keeps track of all the movements and is critical in calculating the total action cost as well as the final path cost.  This will also be critical for the visualization of the different paths that the agent has taken.  
* Goal state = The goal state is the final position that the agent is attempting to find.  Similarly to the initial state, the goal state can be printed using the print statement: print("Goal location:", mh.find_pos(maze, what = "G")).
* Path cost = The path cost represents the final path that the agent has returned as its solution.  Similarly to the total actions, we want this to be as low as possible and is a key factor in determining the successfulness of our model.  I want this value to be as close to the optimal path cost as possible.

Give some estimates for the problem size:

Describe how you would determin these values for a given maze.

* $n$: state space size = To determine the state space size for any given maze, I would first have to analyze the size of the maze and the different state possibilities for each square unit within the maze.  After doing this, I would multiply the height and width values of the size and raise them to the power of the number of possible states per square multiplied again by the height x width.
* $d$: depth of the optimal solution = The depth of the optimal solution can be taken by finding the minimum y-value within the returned path and subtracting that from the maximum y-value in the returned path. 
* $m$: maximum depth of tree = The maximum depth of the tree can be returned by taking the depth of each path the tree reacehes throughout the algorithm using the same method as the depth of the optimal solution.  The only difference in this situation is you want to compare every possible depth and not just the depth of the optimal solution and everytime you reach a depth that is bigger than the current max depth, the current max depth variable is overwritten by the new max depth (this can be done simply using an if-statement).
* $b$: maximum branching factor = The maximum branching factor can be calculated by incrementing the number of times the algorithm has to complete an expand operation and once the algorithm is completed and there are no more expand operations to complete, the maximum branching factor will be solved.

## Task 2: Uninformed search: Breadth-first and depth-first [40 Points]

Implement these search strategies. Follow the pseudocode in the textbook/slides. You can use the tree structure shown above to extract the final path from your solution.

__Notes:__
* You can find maze solving implementations online that use the map to store information. While this is an effective idea for this two-dimensional navigation problem, it typically cannot be used for other search problems. Therefore, follow the textbook and only store information in the tree created during search, and use the `reached` and `frontier` data structures.
* DFS can be implemented using the BFS tree search algorithm and simply changing the order in which the frontier is expanded (this is equivalent to best-first search with path length as the criterion to expand the next node). However, to take advantage of the significantly smaller memory footprint of DFS, you need to implement DFS in a different way without a `reached` data structure and by releasing the memory for nodes that are not needed anymore. 
* If DFS does not use a `reached` data structure, then its cycle checking abilities are limited. Remember, that DFS is incomplete if cycles cannot be prevented. You will see in your experiments that open spaces are a problem.

In [32]:
# Your code goes here

actions = ["Up", "Down", "Left", "Right"]

def expand(maze, node):
    
    global actions
    expandArray = []
    currPos = node.pos
    
    # checks each possible action and adds reachable nodes to array
    for currAction in actions:
        if(currAction == "Up"):
            tempPos1 = (currPos[0],currPos[1]-1)
            if((mh.look(maze,tempPos1) != 'X')):
                addedNode = Node(tempPos1, node, "Up", node.cost+1)
                expandArray.append(addedNode)
        if(currAction == "Down"):
            tempPos2 = (currPos[0],currPos[1]+1)
            if((mh.look(maze,tempPos2) != 'X')):
                addedNode = Node(tempPos2, node, "Down", node.cost+1)
                expandArray.append(addedNode)
        if(currAction == "Left"):
            tempPos3 = (currPos[0]-1,currPos[1])
            if((mh.look(maze,tempPos3) != 'X')):
                addedNode = Node(tempPos3, node, "Left", node.cost+1)
                expandArray.append(addedNode)
        if(currAction == "Right"):
            tempPos4 = (currPos[0]+1,currPos[1])
            if(mh.look(maze,tempPos4) != 'X'):
                addedNode = Node(tempPos4, node, "Right", node.cost+1)
                expandArray.append(addedNode)
    
    return expandArray

In [12]:
def checkArray(array, node):
    
    for element in array:
        if(element.pos == node.pos):
            return False
    
    return True

In [6]:
import numpy as np

def clearMaze(maze):
        
    for location in np.nditer(maze):
        if(location == "."):
            pos = mh.find_pos(maze,".")
            maze[pos] = " "
    
    return maze

In [60]:
from collections import deque

def bfs(maze):
    
    maze = clearMaze(maze)
    totalActions = 0
    frontierQueue = deque()
    reachedArray = []
    # used to track algo statisitics: maxDepth, Max frontier size, Max nodes in memory, and number of nodes expanded
    yCoordinates = []
    maxFrontier = 0
    nodesInMem = 0
    nodesExpanded = 0
    initialPos = mh.find_pos(maze, what = "S")
    node = Node(pos = initialPos, parent = None, action = None, cost = 0) 
    
    if(mh.look(maze, node.pos) == "G"):
        temp = node.get_path_from_root()
        finalPath = [element.pos for element in temp]
        for element in finalPath:
            y = element[1]
            yCoordinates.append(y)
        maxDepth = max(yCoordinates) - min(yCoordinates)
        totalCost = sum([element.cost for element in temp])
        # mh.show_maze(maze, finalPath)
        print("Final path completed in",temp[len(temp)-1].cost,"steps.")
        print("Max depth reached by search is",1,"node.")
        print("Max frontier size is",0,"nodes.")
        print("Max number of nodes in memory is",1,"node.")
        print("Number of nodes expanded is",0,"nodes.")
        return "Success"
    
    frontierQueue.append(node)
    reachedArray.append(node)
    
    # mh.show_maze(maze)
    
    while(len(frontierQueue) != 0):
        totalActions += 1
        tempFront = len(frontierQueue)
        currNode = frontierQueue.popleft()
        if(tempFront > maxFrontier):
            maxFrontier = tempFront
        for element in expand(maze,currNode):
            s = element
            currPath = s.get_path_from_root()
            if(checkArray(reachedArray,element) == True):
                nodesExpanded += 1
            if(len(currPath) > nodesInMem):
                nodesInMem = len(currPath)
            if(mh.look(maze, s.pos) == "G"):
                temp = s.get_path_from_root()
                finalPath = [element.pos for element in temp]
                for element in finalPath:
                    y = element[1]
                    yCoordinates.append(y)
                maxDepth = max(yCoordinates) - min(yCoordinates)
                totalCost = sum([element.cost for element in temp])
                print("Final path completed in",temp[len(temp)-1].cost,"steps.")
                print("Max depth reached by search is",str(maxDepth),"nodes.")
                print("Max frontier size is",str(maxFrontier),"nodes.")
                print("Max number of nodes in memory is",str(nodesInMem),"nodes.")
                print("Number of nodes expanded is",str(nodesExpanded),"nodes.")
                return "Success" 
            if(checkArray(reachedArray, s) == True):
                if(mh.look(maze,s.pos) != "S"):
                    maze[s.pos] = "."
                reachedArray.append(s)
                frontierQueue.append(s)
                # mh.show_maze(maze)
    
    return "Failure"

In [61]:
bfs(maze)

Final path completed in 19 steps.
Max depth reached by search is 12 nodes.
Max frontier size is 8 nodes.
Max number of nodes in memory is 20 nodes.
Number of nodes expanded is 91 nodes.


'Success'

In [38]:
def dfs(maze):
    
    maze = clearMaze(maze)
    totalActions = 0
    frontierQueue = deque()
    # used to track algo statisitics: maxDepth, Max frontier size, Max nodes in memory, and number of nodes expanded
    yCoordinates = []
    maxFrontier = 0
    nodesInMem = 0
    nodesExpanded = 0
    initialPos = mh.find_pos(maze, what = "S")
    node = Node(pos = initialPos, parent = None, action = None, cost = 0) 
    
    if(mh.look(maze, node.pos) == "G"):
        temp = node.get_path_from_root()
        finalPath = [element.pos for element in temp]
        for element in finalPath:
            y = element[1]
            yCoordinates.append(y)
        maxDepth = max(yCoordinates) - min(yCoordinates)
        totalCost = sum([element.cost for element in temp])
        # mh.show_maze(maze, finalPath)
        print("Final path completed in",temp[len(temp)-1].cost,"steps.")
        print("Max depth reached by search is",1,"node.")
        print("Max frontier size is",0,"nodes.")
        print("Max number of nodes in memory is",1,"node.")
        print("Number of nodes expanded is",0,"nodes.")
        return "Success"
    
    frontierQueue.append(node)
    
    while(len(frontierQueue) != 0):
        totalActions += 1
        tempFront = len(frontierQueue)
        currNode = frontierQueue.pop()
        if(tempFront > maxFrontier):
            maxFrontier = tempFront
        for element in expand(maze,currNode):
            s = element
            currPath = s.get_path_from_root()
            nodesExpanded += 1
            if(len(currPath) > nodesInMem):
                nodesInMem = len(currPath)
            if(mh.look(maze, s.pos) == "G"):
                temp = s.get_path_from_root()
                finalPath = [element.pos for element in temp]
                for element in finalPath:
                    y = element[1]
                    yCoordinates.append(y)
                maxDepth = max(yCoordinates) - min(yCoordinates)
                totalCost = sum([element.cost for element in temp])
                # mh.show_maze(maze, finalPath)
                print("Final path completed in",temp[len(temp)-1].cost,"steps.")
                print("Max depth reached by search is",str(maxDepth),"nodes.")
                print("Max frontier size is",str(maxFrontier),"nodes.")
                print("Max number of nodes in memory is",str(nodesInMem),"nodes.")
                print("Number of nodes expanded is",str(nodesExpanded),"nodes.")
                return "Success" 
            currPath = currNode.get_path_from_root()
            if(checkArray(currPath, s) == True):
                if(mh.look(maze,s.pos) != "S"):
                    maze[s.pos] = "."
                frontierQueue.append(s)
                # mh.show_maze(maze)
    
    return "Failure"

In [39]:
dfs(maze)

Final path completed in 37 steps.
Max depth reached by search is 19 nodes.
Max frontier size is 7 nodes.
Max number of nodes in memory is 46 nodes.
Number of nodes expanded is 158 nodes.


'Success'

How does BFS and DFS deal with loops (cycles)?

My BFS and DFS algorithms each deal with cycles in a very different way.  The BFS algorithm makes use of a "reached" datastructure which is nothing more than an array which contains all of the different nodes that my algorithm has already visited.  The algorithm will check this array and make sure that it does not contain each sequential node before that node is added to the "frontier" datastructure and actually visited by the algorithm.  This prevents the algorithm from repeatedly returning to any given node and forces it to explore all unexplored nodes searching for the goal state.  On the contrary, my DFS algorithm does not use a "reached" array in order to make use of its smaller memory footprint.  Instead, it uses the function get_path_from_root() function which is already built into the node structure detailed in the code at the top of this document.  This function returns a list of all of the nodes from the current node to the start position and I use it to make sure that my algorithm is not going to squares that it has already checked.  Similarly to the reached structure in the DFS algorithm, the BFS algorithm checks the list returned from the get_path_from_root() function before adding a new position to the frontier datastructure.

Are your implementations complete and optimal? Explain why. What is the time and space complexity of each of **your** implementations?

Each of the implementations above are complete, but cannot be gauranteed to be optimal.  As both breadth-first and depth-first rather exhaustively search the environment until they reach the goal state, they can be gauranteed to find the goal state eventually if there is one and it is reachable.  However, each of these algorithms (in my current implementation) return the first path that they find and therefore cannot be gauranteed to find the most optimal solution every time.  Breadth first search has both a space and time complexity of O(b^d).  Similarly, depth-first search has a time complexity of O(b^m), but is more efficient than depth-first search in its space complexity with breadth-first search having a space complexity of O(bm).

## Task 3: Informed search: Implement greedy best-first search and A* search  [20 Points]

You can use the map to estimate the distance from your current position to the goal using the Manhattan distance (see https://en.wikipedia.org/wiki/Taxicab_geometry) as a heuristic function. Both algorithms are based on Best-First search which requires only a small change from the BFS algorithm you have already implemented (see textbook/slides). 

In [42]:
# Your code goes here
def manhattanDist(currPos, goalPos):
    solution = (abs(currPos[0]-goalPos[0]) + abs(currPos[1]-goalPos[1]))
    return solution

In [48]:
from queue import PriorityQueue
import heapdict

def gbfs(maze):
    
    maze = clearMaze(maze)
    totalActions = 0
    # used to track algo statisitics: maxDepth, Max frontier size, Max nodes in memory, and number of nodes expanded
    yCoordinates = []
    maxFrontier = 0
    nodesInMem = 0
    nodesExpanded = 0
    initialPos = mh.find_pos(maze, what = "S")
    goalPos = mh.find_pos(maze, what = "G")
    node = Node(pos = initialPos, parent = None, action = None, cost = 0) 
    
    currManDist = manhattanDist(initialPos,goalPos)
    
    # frontierArray stores the manhattan distance at each node
    # heapdict stores nodes as tuples, first item is node, second is manhattan distance
    frontierArray = heapdict.heapdict()
    frontierArray[node] = currManDist
    
    reachedArray = []
    reachedArray.append(node)
    
    if(mh.look(maze, node.pos) == "G"):
        temp = node.get_path_from_root()
        finalPath = [element.pos for element in temp]
        for element in finalPath:
            y = element[1]
            yCoordinates.append(y)
        maxDepth = max(yCoordinates) - min(yCoordinates)
        totalCost = sum([element.cost for element in temp])
        # mh.show_maze(maze, finalPath)
        print("Final path completed in",temp[len(temp)-1].cost,"steps.")
        print("Max depth reached by search is",1,"node.")
        print("Max frontier size is",0,"nodes.")
        print("Max number of nodes in memory is",1,"node.")
        print("Number of nodes expanded is",0,"nodes.")
        return "Success"
    
    while(len(frontierArray) != 0):
        totalActions += 1
        tempFront = len(frontierArray)
        temp = frontierArray.popitem()
        if(tempFront > maxFrontier):
            maxFrontier = tempFront
        currNode = temp[0]
        currManDist = temp[1]
        for element in expand(maze, currNode):
            currPath = element.get_path_from_root()
            nodesExpanded += 1
            if(len(currPath) > nodesInMem):
                nodesInMem = len(currPath)
            if(mh.look(maze, element.pos) == "G"):
                temp = element.get_path_from_root()
                finalPath = [element.pos for element in temp]
                for element in finalPath:
                    y = element[1]
                    yCoordinates.append(y)
                maxDepth = max(yCoordinates) - min(yCoordinates)
                totalCost = sum([element.cost for element in temp])
                # mh.show_maze(maze, finalPath)
                print("Final path completed in",temp[len(temp)-1].cost,"steps.")
                print("Max depth reached by search is",str(maxDepth),"nodes.")
                print("Max frontier size is",str(maxFrontier),"nodes.")
                print("Max number of nodes in memory is",str(nodesInMem),"nodes.")
                print("Number of nodes expanded is",str(nodesExpanded),"nodes.")
                return "Success"
            if(checkArray(reachedArray,element) == True):
                if(mh.look(maze,element.pos) != "S"):
                    maze[element.pos] = "."
                currManDist = manhattanDist(element.pos,goalPos)
                reachedArray.append(element)
                frontierArray[element] = currManDist
                # mh.show_maze(maze)
                
    return "Failure"

In [49]:
gbfs(maze)

Final path completed in 29 steps.
Max depth reached by search is 11 nodes.
Max frontier size is 5 nodes.
Max number of nodes in memory is 30 nodes.
Number of nodes expanded is 80 nodes.


'Success'

In [46]:
def aStar(maze):
    
    maze = clearMaze(maze)
    totalActions = 0
    # used to track algo statisitics: maxDepth, Max frontier size, Max nodes in memory, and number of nodes expanded
    yCoordinates = []
    maxFrontier = 0
    nodesInMem = 0
    nodesExpanded = 0
    initialPos = mh.find_pos(maze, what = "S")
    goalPos = mh.find_pos(maze, what = "G")
    node = Node(pos = initialPos, parent = None, action = None, cost = 0) 
    
    currCost = manhattanDist(initialPos,goalPos) + node.cost
    
    # frontierArray stores the manhattan distance at each node
    # heapdict stores nodes as tuples, first item is node, second is heuristic cost
    frontierArray = heapdict.heapdict()
    frontierArray[node] = currCost
    
    reachedArray = []
    reachedArray.append(node)
    
    if(mh.look(maze, node.pos) == "G"):
        temp = node.get_path_from_root()
        finalPath = [element.pos for element in temp]
        for element in finalPath:
            y = element[1]
            yCoordinates.append(y)
        maxDepth = max(yCoordinates) - min(yCoordinates)
        totalCost = sum([element.cost for element in temp])
        # mh.show_maze(maze, finalPath)
        print("Final path completed in",temp[len(temp)-1].cost,"steps.")
        print("Max depth reached by search is",1,"node.")
        print("Max frontier size is",0,"nodes.")
        print("Max number of nodes in memory is",1,"node.")
        print("Number of nodes expanded is",0,"nodes.")
        return "Success"
    
    while(len(frontierArray) != 0):
        totalActions += 1
        tempFront = len(frontierArray)
        temp = frontierArray.popitem()
        if(tempFront > maxFrontier):
            maxFrontier = tempFront
        currNode = temp[0]
        currCost = temp[1]
        for element in expand(maze, currNode):
            currPath = element.get_path_from_root()
            nodesExpanded += 1
            if(len(currPath) > nodesInMem):
                nodesInMem = len(currPath)
            if(mh.look(maze, element.pos) == "G"):
                temp = element.get_path_from_root()
                finalPath = [element.pos for element in temp]
                for element in finalPath:
                    y = element[1]
                    yCoordinates.append(y)
                maxDepth = max(yCoordinates) - min(yCoordinates)
                totalCost = sum([element.cost for element in temp])
                # mh.show_maze(maze, finalPath)
                print("Final path completed in",temp[len(temp)-1].cost,"steps.")
                print("Max depth reached by search is",str(maxDepth),"nodes.")
                print("Max frontier size is",str(maxFrontier),"nodes.")
                print("Max number of nodes in memory is",str(nodesInMem),"nodes.")
                print("Number of nodes expanded is",str(nodesExpanded),"nodes.")
                return "Success"
                return "Success"
            if(checkArray(reachedArray,element) == True):
                if(mh.look(maze,element.pos) != "S"):
                    maze[element.pos] = "."
                currCost = manhattanDist(element.pos,goalPos) + element.cost
                reachedArray.append(element)
                frontierArray[element] = currCost
                # mh.show_maze(maze)
                
    return "Failure"

In [47]:
aStar(maze)

Final path completed in 19 steps.
Max depth reached by search is 12 nodes.
Max frontier size is 8 nodes.
Max number of nodes in memory is 20 nodes.
Number of nodes expanded is 101 nodes.


'Success'

Are your implementations complete and optimal? What is the time and space complexity?

Both greedy best-first search and AStar search are complete and will ultimately find the goal state no matter what.  Greedy best-first search does not always return the optimal solution; however, AStar does making it the most successful of the four algorithm implementations that I created.  Greedy best-first search has a worst case time complexity of O(b^m) and a best case complexity of O(bm) with the space complexity being the same as the time complexity.  On the other hand, AStar search has a worst case time complexity of O(b^m) which is similar to greedy best-first search and also has the same space complexity as its time complexity

## Task 4: Comparison and discussion [20 Points] 

Run experiments to compare the implemented algorithms.

How to deal with issues:

* Your implementation returns unexpected results: Try to debug and fix the code. Visualizing the maze, the current path and the frontier after every step is very helpful. If the code still does not work, then mark the result with an asterisk (*) and describe the issue below the table.

* Your implementation cannot consistently solve a specific maze and ends up in an infinite loop:
    Debug. If it is a shortcoming of the algorithm/implementation, then put "N/A*" in the results table and describe why this is happening.

In [50]:
# Add code
# Medium Maze

with open("medium_maze.txt", "r") as f:
    maze_str = f.read()
    
mMaze = mh.parse_maze(maze_str)

print("DFS:")
dfs(mMaze)
print("BFS:")
bfs(mMaze)
print("GBFS:")
gbfs(mMaze)
print("A*:")
aStar(mMaze)

DFS:
Final path completed in 234 steps.
Max depth reached by search is 33 nodes.
Max frontier size is 13 nodes.
Max number of nodes in memory is 235 nodes.
Number of nodes expanded is 553 nodes.
BFS:
Final path completed in 68 steps.
Max depth reached by search is 33 nodes.
Max frontier size is 8 nodes.
Max number of nodes in memory is 69 nodes.
Number of nodes expanded is 269 nodes.
GBFS:
Final path completed in 74 steps.
Max depth reached by search is 33 nodes.
Max frontier size is 4 nodes.
Max number of nodes in memory is 75 nodes.
Number of nodes expanded is 159 nodes.
A*:
Final path completed in 68 steps.
Max depth reached by search is 33 nodes.
Max frontier size is 8 nodes.
Max number of nodes in memory is 69 nodes.
Number of nodes expanded is 452 nodes.


'Success'

In [51]:
# Large Maze

with open("large_maze.txt", "r") as f:
    maze_str = f.read()
    
lMaze = mh.parse_maze(maze_str)

print("DFS:")
dfs(lMaze)
print("BFS:")
bfs(lMaze)
print("GBFS:")
gbfs(lMaze)
print("A*:")
aStar(lMaze)

DFS:
Final path completed in 210 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 32 nodes.
Max number of nodes in memory is 211 nodes.
Number of nodes expanded is 890 nodes.
BFS:
Final path completed in 210 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 8 nodes.
Max number of nodes in memory is 211 nodes.
Number of nodes expanded is 622 nodes.
GBFS:
Final path completed in 210 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 21 nodes.
Max number of nodes in memory is 211 nodes.
Number of nodes expanded is 951 nodes.
A*:
Final path completed in 210 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 12 nodes.
Max number of nodes in memory is 211 nodes.
Number of nodes expanded is 1094 nodes.


'Success'

In [52]:
# Loops Maze

with open("loops_maze.txt", "r") as f:
    maze_str = f.read()
    
lmaze = mh.parse_maze(maze_str)

print("DFS:")
dfs(lmaze)
print("BFS:")
bfs(lmaze)
print("GBFS:")
gbfs(lmaze)
print("A*:")
aStar(lmaze)

DFS:
Final path completed in 41 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 17 nodes.
Max number of nodes in memory is 42 nodes.
Number of nodes expanded is 129 nodes.
BFS:
Final path completed in 23 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 7 nodes.
Max number of nodes in memory is 24 nodes.
Number of nodes expanded is 71 nodes.
GBFS:
Final path completed in 35 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 9 nodes.
Max number of nodes in memory is 36 nodes.
Number of nodes expanded is 135 nodes.
A*:
Final path completed in 23 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 9 nodes.
Max number of nodes in memory is 24 nodes.
Number of nodes expanded is 135 nodes.


'Success'

In [54]:
# Open Maze

with open("open_maze.txt", "r") as f:
    maze_str = f.read()
    
oMaze = mh.parse_maze(maze_str)

print("DFS: N/A")
# dfs(oMaze)
print("BFS:")
bfs(oMaze)
print("GBFS:")
gbfs(oMaze)
print("A*:")
aStar(oMaze)

DFS: N/A
BFS:
Final path completed in 54 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 23 nodes.
Max number of nodes in memory is 55 nodes.
Number of nodes expanded is 683 nodes.
GBFS:
Final path completed in 68 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 67 nodes.
Max number of nodes in memory is 69 nodes.
Number of nodes expanded is 283 nodes.
A*:
Final path completed in 54 steps.
Max depth reached by search is 34 nodes.
Max frontier size is 113 nodes.
Max number of nodes in memory is 55 nodes.
Number of nodes expanded is 454 nodes.


'Success'

In [55]:
# Wall Maze

with open("wall_maze.txt", "r") as f:
    maze_str = f.read()
    
wMaze = mh.parse_maze(maze_str)

print("DFS:")
dfs(wMaze)
print("BFS:")
bfs(wMaze)
print("GBFS:")
gbfs(wMaze)
print("A*:")
aStar(wMaze)

DFS:
Final path completed in 26 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 30 nodes.
Max number of nodes in memory is 27 nodes.
Number of nodes expanded is 89 nodes.
BFS:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 11 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 88 nodes.
GBFS:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 27 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 54 nodes.
A*:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 27 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 54 nodes.


'Success'

In [56]:
# Empty Maze

with open("empty_maze.txt", "r") as f:
    maze_str = f.read()
    
eMaze = mh.parse_maze(maze_str)

print("DFS:")
dfs(eMaze)
print("BFS:")
bfs(eMaze)
print("GBFS:")
gbfs(eMaze)
print("A*:")
aStar(eMaze)

DFS:
Final path completed in 54 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 59 nodes.
Max number of nodes in memory is 55 nodes.
Number of nodes expanded is 202 nodes.
BFS:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 12 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 95 nodes.
GBFS:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 27 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 54 nodes.
A*:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 27 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 54 nodes.


'Success'

In [57]:
# Empty Maze 2

with open("empty_2_maze.txt", "r") as f:
    maze_str = f.read()
    
e2Maze = mh.parse_maze(maze_str)

print("DFS:")
dfs(e2Maze)
print("BFS:")
bfs(e2Maze)
print("GBFS:")
gbfs(e2Maze)
print("A*:")
aStar(e2Maze)

DFS:
Final path completed in 72 steps.
Max depth reached by search is 8 nodes.
Max frontier size is 65 nodes.
Max number of nodes in memory is 73 nodes.
Number of nodes expanded is 260 nodes.
BFS:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 12 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 95 nodes.
GBFS:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 27 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 53 nodes.
A*:
Final path completed in 14 steps.
Max depth reached by search is 7 nodes.
Max frontier size is 27 nodes.
Max number of nodes in memory is 15 nodes.
Number of nodes expanded is 53 nodes.


'Success'

__Small maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     19    |         91          |        12      |            20            |        8          |
| DFS       |     37    |        158          |        19      |            46            |        7          |
| GBS       |     29    |         80          |        11      |            30            |        5          |
| A*        |     19    |        101          |        12      |            20            |        8          |

__Medium Maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     68   |         269         |        33      |            69           |        8         |
| DFS       |     234    |        553          |        33      |            235            |        13          |
| GBS       |     74    |         159          |        33      |            75            |        4          |
| A*        |     68    |        452          |        33      |            69            |        8          |

__Large Maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     210    |         622          |        34      |            211            |        8          |
| DFS       |     210    |        890          |        34      |            211           |        32          |
| GBS       |     210    |         951          |        34      |            211            |        21          |
| A*        |     210    |        1094          |        34      |            211            |        12          |

__Loops Maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     23    |         71          |        7      |            24            |        7          |
| DFS       |     41    |        129          |        7      |            42            |        17          |
| GBS       |     35    |         135          |        7      |            36            |        9          |
| A*        |     23    |        135          |        7      |            24            |        9          |


__Open_Maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     54    |         683          |        34      |            55            |        23          |
| DFS       |     N/A    |        N/A          |        N/A      |            N/A            |        N/A          |
| GBS       |     68    |         283          |        34      |            69            |        67          |
| A*        |     54    |        454          |        34      |            55            |        113          |

__Wall Maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     14    |         88          |        7      |            15            |        11          |
| DFS       |     26    |        89          |        7      |            27           |        30          |
| GBS       |     14    |         54          |        7      |            15            |        27         |
| A*        |     14    |        54          |        7      |            15           |        27       |


__Empty Maze__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     14    |         95          |        7      |            15            |        12          |
| DFS       |     54    |        202          |        7      |            55            |        59         |
| GBS       |     14    |         54          |        7      |            15            |        27         |
| A*        |     14    |        54          |        7      |            15            |        27        |

__Empty Maze 2__

| algorithm | path cost | # of nodes expanded | max tree depth | max # of nodes in memory | max frontier size |
|-----------|-----------|---------------------|----------------|--------------------------|-------------------|
| BFS       |     14    |         95          |        7      |            15            |        12          |
| DFS       |     72    |        260          |        8      |            73            |        65         |
| GBS       |     14    |         53          |        7      |            15            |        27         |
| A*        |     14    |        53          |        7      |            15            |        27        |

Present the results as using charts (see [Python Code Examples/charts and tables](../Python_Code_Examples/charts_and_tables.ipynb)). 

Discuss the most important lessons you have learned from implementing the different search strategies. 

Through the implementation of breadth-first, depth-first, greedy best-first, and A* search, as well as their application towards solving the eight different mazes above, I have learned a great deal about these different search strategies.  First, I was amazed to find that the implementation of each of these algorithms, despite their fundamental differences, were incredibly similar except for a few key differences that gave each algorithm its unique characteristics.  In addition to the many similarities in implementation, I was also surprised to learn how similar the performance of breadth-first search was to A* search on many of the different mazes.  I had always thought that A* was a far superior searching strategy, but on many of the different maze environments this was not necessarily the case.  On the other hand, I was also very surprised to see how poorly depth-first search performed with it constantly ranking amongst the worst performing algorithms on most of the mazes and not even being able to reach a solution on the open maze.  I believe that the main reason depth-first search struggled so much with the open maze was a problem with my expand function as it did not have any logic to prevent the returned nodes from being out of the bounds of the maze allowing depth-first search to expand infinitely downward far outside the bounds of the maze.  I tried to fix this in numerous ways and assumed the fix would be relatively easy to implement; however, I could not find a solution that didn't adversely effect the rest of the algorithms and ultimately decided to give up on that.

## Graduate student advanced task: Multiple goals [10 Points]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus Points].

Create a few mazes with multiple goals by adding one or two more goals to the medium size maze.
Solve the maze with your implementations for DFS, BFS, and implement in addition IDS (iterative deepening search using DFS). 

Run experiments to show which implementations find the optimal solution and which do not. Discuss why that is the case.

In [ ]:
# Your code/answer goes here

## More advanced tasks to think about

Instead of defining each square as a state, use only intersections as states. Now the storage requirement is reduced, but the path length between two intersections can be different. If we use total path length measured as the number of squares as path cost, how can we make sure that BFS and iterative deepening search is optimal? Change the code to do so.

In [ ]:
# Your code/answer goes here

Modify your A* search to add weights (see text book) and explore how different weights influence the result.

In [ ]:
# Your code/answer goes here

What happens if the agent does not know the layout of the maze in advance (i.e., faces an unkown, only partially observable environment)? How does the environment look then (PEAS description)? How would you implement a rational agent to solve the maze? What if the agent still has a GPS device to tell the distance to the goal?

In [ ]:
# Your code/answer goes here